In [ ]:
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from scipy import interpolate
import numpy as np
import json

# Save simulation results

This function saves all the simulation results in a dictionary. Furthermore, since the $mpc.make\_step()$ function stores all the computed optimal control inputs every $t\_step$, although the one applied to the process is updated only every $15$ minutes, the function takes care of updating the trajectory of the inputs with those actually used.

In [ ]:
def save_results(mpc, avg_period, t_step):

    states = {
        'mRNA_LacI': [x[0] for x in mpc.data['_x', 'mRNA_LacI'].tolist()],
        'mRNA_TetR': [x[0] for x in mpc.data['_x', 'mRNA_TetR'].tolist()],
        'LacI': [x[0] for x in mpc.data['_x', 'LacI'].tolist()],
        'TetR': [x[0] for x in mpc.data['_x', 'TetR'].tolist()],
        'v1': [x[0] for x in mpc.data['_x', 'v1'].tolist()],
        'v2': [x[0] for x in mpc.data['_x', 'v2'].tolist()],
        
    }

    u = [x[0] for x in mpc.data['_u', 'phi'].tolist()]

    for i in range(len(u)):
        if i % 15 != 0:
            u[i] = u[i-1]

    inputs = {
        'phi': u,
        'aTc': [x * 35 for x in u],
        'IPTG': [(1 - x) * 0.35 for x in u]
    }
    
    # Get average trajectory every 240 minutes (14400 s)
    # Get samples every 5 minutes (300 s)
    # Samples in 240 minutes = 240/5 = 48 samples
    avg_samples_range = int(avg_period/t_step)

    avg_LacI = [np.mean(states['LacI'][x:x + avg_samples_range])
                for x in range(0, len(states['LacI']), avg_samples_range)]

    avg_TetR = [np.mean(states['TetR'][x:x + avg_samples_range])
                for x in range(0, len(states['TetR']), avg_samples_range)]

    avg_trajectory = {
        'LacI': avg_LacI,
        'TetR': avg_TetR
    }

    cost = [x[0] for x in mpc.data['_aux', 'cost'].tolist()]
    time = [x[0] for x in mpc.data['_time'].tolist()]

    data = {
        'states': states,
        'inputs': inputs,
        'avg_traj': avg_trajectory,
        'cost': cost,
        'time': time,
        'ISE': 0,
        'ITAE': 0
    }

    return data


# Export simulation results

This function exports all the simulation results in a json file.

In [ ]:
def export_results(data, type, name, mode):

    f = open('./data/' + type + '/' + name + '.json', mode)
    data_json = json.dumps(data)
    f.write(data_json)
    f.close()


# Compute the performance metrics

This function compute the performance metrics such as:

\begin{equation}
    \begin{split}
        &\text{ITAE} = \int_{t_0}^T \tau |\bar{e}(\tau)| \; d\tau\\
        &\text{ISE} =  \int_{t_0}^T \bar{e}(\tau)^2 \; d\tau
    \end{split}
\end{equation}
where $\bar{e}(\tau)$ is:

\begin{equation}
    \bar{e}(\tau) = \left| \left[\frac{\bar{LacI} - LacI_{ref}}{LacI_{ref}},  \frac{\bar{TetR} - TetR_{ref}}{TetR_{ref}}\right] \right|
\end{equation}
and $\bar{LacI}$ and $\bar{TetR}$ are the moving averages of $LacI$ and $TetR$ over a window of $240m$ width.

In [ ]:
def compute_performance_metrics(data, total_time, t_step, avg_period):

    avg_x = np.arange(0, total_time, avg_period)

    e3_bar = np.array([(x - 750) / 750 for x in data['avg_traj']['LacI']])
    e4_bar = np.array([(x - 300) / 300 for x in data['avg_traj']['TetR']])

    fe3_bar = interpolate.interp1d(avg_x, e3_bar, fill_value="extrapolate")
    e3_bar = fe3_bar(np.arange(0, total_time, t_step))
    fe4_bar = interpolate.interp1d(avg_x, e4_bar, fill_value="extrapolate")
    e4_bar = fe4_bar(np.arange(0, total_time, t_step))

    e_bar = np.array([np.linalg.norm([e3_bar[i], e4_bar[i]])
                      for i in range(len(e3_bar))])

    ISE = np.sum((e_bar)**2)
    ITAE = np.sum([np.abs(e_bar[i])*(60*(i+1)) for i in range(len(e_bar))])
    
    data['ISE'] = ISE
    data['ITAE'] = ITAE

    return ISE, ITAE

# Plot simulation results

This function plot all the simulation results.

In [ ]:
def plot_results(data, total_time, avg_period):

    fig_x = 20
    fig_y = 10

    x_ticks = [x for x in range(0, total_time + 1, int(total_time/6))]
    x_ticks_label = [int(x/60) for x in x_ticks]
    avg_x = np.arange(0, total_time, avg_period)

    plt.rcParams['axes.grid'] = True
    plt.rcParams['font.size'] = 14

    # -------------------- Proteins -------------------- #
    # --- LacI --- #
    figure_proteins, axes = plt.subplots(
        2, sharex=True, figsize=(fig_x, fig_y))    

    axes[0].set_ylabel('')
    axes[0].set_title('LacI')

    line_LacI, = axes[0].plot(data['time'], data['states']['LacI'], color='b')
    #line_avg_LacI, = axes[0].plot(avg_x, data['avg_traj']['LacI'], color='b', linestyle='--')
    line_ref_LacI, = axes[0].plot(
        data['time'], 750*np.ones(len(data['time'])), color='k', linestyle='--')
    axes[0].legend(['LacI', 'Avg LacI traj', 'LacI target'], loc='upper right')

    # --- TetR --- #
    axes[1].set_ylabel('')
    axes[1].set_title('TetR')
    line_TetR, = axes[1].plot(data['time'], data['states']['TetR'], color='m')
    #line_avg_TetR, = axes[1].plot(avg_x, data['avg_traj']['TetR'], color='m', linestyle='--')
    line_ref_TetR, = axes[1].plot(
        data['time'], 300*np.ones(len(data['time'])), color='k', linestyle='--')
    axes[1].legend(['TetR', 'Avg TetR traj', 'TetR target'], loc='upper right')
    axes[1].set_xlabel('time [min]')
    axes[1].set_xticks(x_ticks)
    axes[1].set_xticklabels(x_ticks_label, rotation=30)

    # -------------------- mRNAs -------------------- #
    # --- mRNA LacI --- #
    figure_mRNAs, axes1 = plt.subplots(2, sharex=True, figsize=(fig_x, fig_y))
    axes1[0].set_ylabel('')
    axes1[0].set_title('mRNA_LacI')
    line_mRNA_LacI, = axes1[0].plot(data['time'], data['states']
                                    ['mRNA_LacI'], color='b')
    axes1[0].legend(['mRNA LacI'], loc='upper right')

    # --- mRNA TetR --- #
    axes1[1].set_ylabel('')
    axes1[1].set_title('mRNA_TetR')
    line_mRNA_TetR, = axes1[1].plot(data['time'], data['states']
                                    ['mRNA_TetR'], color='m')
    axes1[1].legend(['mRNA TetR'], loc='upper right')
    axes1[1].set_xlabel('time [min]')
    axes1[1].set_xticks(x_ticks)
    axes1[1].set_xticklabels(x_ticks_label, rotation=30)

    # -------------------- Internal inducers concentrations -------------------- #
    # --- v1 --- #
    figure_int_inducers, axes2 = plt.subplots(
        2, sharex=True, figsize=(fig_x, fig_y))
    axes2[0].set_ylabel('')
    axes2[0].set_title('$v1$')
    line_int_aTc, = axes2[0].plot(
        data['time'], data['states']['v1'], color='y')
    axes2[0].legend(['$v1$'], loc='upper right')

    # --- v2 --- #
    axes2[1].set_ylabel('')
    axes2[1].set_title('$v2$')
    line_int_IPTG, = axes2[1].plot(
        data['time'], data['states']['v2'], color='r')
    axes2[1].legend(['$v2$'], loc='upper right')
    axes2[1].set_xlabel('time [min]')
    axes2[1].set_xticks(x_ticks)
    axes2[1].set_xticklabels(x_ticks_label, rotation=30)

    # -------------------- External inducers concentrations -------------------- #
    # --- aTc --- #
    figure_inducers, axes2 = plt.subplots(
        2, sharex=True, figsize=(fig_x, fig_y))
    axes2[0].set_ylabel('')
    axes2[0].set_title('aTc')
    line_aTc, = axes2[0].plot(data['time'], data['inputs']['aTc'], color='y')
    axes2[0].legend(['aTc'], loc='upper right')

    # --- IPTG --- #
    axes2[1].set_ylabel('')
    axes2[1].set_title('IPTG')
    line_IPTG, = axes2[1].plot(data['time'], data['inputs']['IPTG'], color='r')
    axes2[1].legend(['IPTG'], loc='upper right')
    axes2[1].set_xticks(x_ticks)
    axes2[1].set_xticklabels(x_ticks_label, rotation=30)
  
    # -------------------- Cost -------------------- #
    figure_cost, axes4 = plt.subplots(1, figsize=(fig_x, fig_y/2))
    axes4.set_ylabel('')
    axes4.set_title('Cost')
    line_cost, = axes4.plot(data['time'], data['cost'], color='g')
    axes4.legend(['Cost'], loc='upper right')
    axes4.set_xlabel('time [min]')
    axes4.set_xticks(x_ticks)
    axes4.set_xticklabels(x_ticks_label, rotation=30)

    # -------------------- Input -------------------- #
    figure_input, axes5 = plt.subplots(1, figsize=(fig_x, fig_y/2))
    axes5.set_ylabel('')
    axes5.set_title('Phi')
    line_input, = axes5.plot(data['time'], data['inputs']['phi'], color='b')
    axes5.legend(['Phi'], loc='upper right')
    axes5.set_xlabel('time [min]')
    axes5.set_xticks(x_ticks)
    axes5.set_xticklabels(x_ticks_label, rotation=30)

    figures = np.array([figure_proteins, figure_mRNAs,
                       figure_int_inducers, figure_inducers, figure_cost, figure_input])
    lines = np.array([line_LacI, #line_avg_LacI, 
                        line_ref_LacI, line_TetR, #line_avg_TetR, 
                     line_ref_TetR,line_mRNA_LacI, line_mRNA_TetR, line_int_aTc, line_int_IPTG, line_aTc, line_IPTG, line_cost, line_input])

    return figures, lines


# Update functions for animation

In [ ]:
def update_mRNA(i, data, lines):
    lines[0].set_data(data['time'][:i], data['states']['mRNA_LacI'][:i])
    lines[1].set_data(data['time'][:i], data['states']['mRNA_TetR'][:i])
    return lines


def update_protein(i, data, lines, avg_x):
    lines[0].set_data(data['time'][:i], data['states']['LacI'][:i])
    lines[1].set_data(avg_x[:i], data['avg_traj']['LacI'][:i])
    lines[2].set_data(data['time'][:i], 750*np.ones(len(data['time']))[:i])
    lines[3].set_data(data['time'][:i], data['states']['TetR'][:i])
    lines[4].set_data(avg_x[:i], data['avg_traj']['TetR'][:i])
    lines[5].set_data(data['time'][:i], 300*np.ones(len(data['time']))[:i])
    return lines


def update_internal_inducers(i, data, lines):
    lines[0].set_data(data['time'][:i], data['states']['v1'][:i])
    lines[1].set_data(data['time'][:i], data['states']['v2'][:i])
    return lines


def update_external_inducers(i, data, lines):
    lines[0].set_data(data['time'][:i], data['inputs']['aTc'][:i])
    lines[1].set_data(data['time'][:i], data['inputs']['IPTG'][:i])
    return lines


def update_cost(i, data, lines):
    lines[0].set_data(data['time'][:i], data['cost'][:i])
    return lines

def update_input(i, data, lines):
    lines[0].set_data(data['time'][:i], data['inputs']['phi'][:i])
    return lines

# Animate simulation results

This function animate simulation plots and export them as $.mp4$ files.

In [ ]:
def animate_results(type, name, funct, figures, fargs, steps):
    anim = animation.FuncAnimation(figures, funct, fargs=fargs, frames=steps)
    anim.save('./simulations/' + type + '/' + name + '.mp4', fps=60)